In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

import geopandas as gpd

from scipy.interpolate import interp1d
from statsmodels.tsa.vector_ar.var_model import VAR


output_dir = '../data/curated/'


<h3> Preparation for forecasting </h3>

In [2]:
properties_df = pd.read_csv("../data/curated/properties_processed.csv", index_col=0)
properties_df['Postcode'] = properties_df['Postcode'].astype(str)

# read in census dataframes
census_df = pd.read_csv("../data/curated/census_data.csv")
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")

In [3]:
def convert_census_to_postcode(census_df, sa2_postcode_map, agg_func):
    ''' Inputs census data as indexed by SA2 and converts it to postcode through aggregation
    '''

    census_df_postcode = sa2_postcode_map.merge(census_df, on='sa2_2021').drop('sa2_2021', axis=1)
    census_df_postcode = census_df_postcode[census_df_postcode['postcode_2021'] >= 3000]

    census_df_postcode_agg = census_df_postcode.groupby('postcode_2021').agg(
        tot_population_11 = pd.NamedAgg(column='Tot_persons_C11_P', aggfunc=sum),
        tot_population_16 = pd.NamedAgg(column='Tot_persons_C16_P', aggfunc=sum),
        tot_population_21 = pd.NamedAgg(column='Tot_persons_C21_P', aggfunc=sum),
        avg_med_mortg_rep_11 = pd.NamedAgg(column='Med_mortg_rep_mon_C2011', aggfunc=agg_func),
        avg_med_mortg_rep_16 = pd.NamedAgg(column='Med_mortg_rep_mon_C2016', aggfunc=agg_func),
        avg_med_mortg_rep_21 = pd.NamedAgg(column='Med_mortg_rep_mon_C2021', aggfunc=agg_func),
        avg_med_person_inc_11 = pd.NamedAgg(column='Med_person_inc_we_C2011', aggfunc=agg_func),
        avg_med_person_inc_16 = pd.NamedAgg(column='Med_person_inc_we_C2016', aggfunc=agg_func),
        avg_med_person_inc_21 = pd.NamedAgg(column='Med_person_inc_we_C2021', aggfunc=agg_func),
        avg_med_rent_16 = pd.NamedAgg(column='Med_rent_weekly_C2011', aggfunc=agg_func),
        avg_med_rent_11 = pd.NamedAgg(column='Med_rent_weekly_C2016', aggfunc=agg_func),
        avg_med_rent_21 = pd.NamedAgg(column='Med_rent_weekly_C2021', aggfunc=agg_func),
        avg_med_hh_inc_16 = pd.NamedAgg(column='Med_tot_hh_inc_wee_C2011', aggfunc=agg_func),
        avg_med_hh_inc_11 = pd.NamedAgg(column='Med_tot_hh_inc_wee_C2016', aggfunc=agg_func),
        avg_med_hh_inc_21 = pd.NamedAgg(column='Med_tot_hh_inc_wee_C2021', aggfunc=agg_func),
        tot_avg_hh_size_16 = pd.NamedAgg(column='Average_hh_size_C2011', aggfunc=agg_func),
        tot_avg_hh_size_11 = pd.NamedAgg(column='Average_hh_size_C2016', aggfunc=agg_func),
        tot_avg_hh_size_21 = pd.NamedAgg(column='Average_hh_size_C2021', aggfunc=agg_func),
    ).reset_index()

    return census_df_postcode_agg

In [4]:
mean_no_zero = lambda lst: round(np.mean([x for x in lst if x > 0]), 2)
census_df_postcode_agg = convert_census_to_postcode(census_df, sa2_postcode_map, mean_no_zero)

Median Rental Prices

In [5]:
median_rent = properties_df.groupby(["Postcode"])["Cost"].median()


In [6]:
median_rent.to_csv(f'{output_dir}median_rental_postcode.csv')

In [7]:
census_df_postcode_agg

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,3990,35361,38876,46703,1405.00,1437.67,1635.33,488.67,552.00,704.00,209.00,244.67,299.33,970.67,1108.00,1453.67,2.47,2.47,2.47
690,3991,20031,22228,26792,1300.00,1300.00,1467.00,466.00,496.00,618.00,230.00,244.00,298.00,853.00,909.00,1154.00,2.20,2.20,2.20
691,3992,20031,22228,26792,1300.00,1300.00,1467.00,466.00,496.00,618.00,230.00,244.00,298.00,853.00,909.00,1154.00,2.20,2.20,2.20
692,3995,28219,30958,36238,1222.00,1235.00,1383.50,450.50,497.00,618.50,200.00,222.00,274.00,823.00,910.50,1132.00,2.25,2.20,2.15


In [19]:
i = 0 
impute_features = ['tot_population', 'avg_med_mortg_rep', 'avg_med_person_inc', 'avg_med_rent', 'avg_med_hh_inc', 'tot_avg_hh_size']
init_years = [11, 16, 21]
new_years = list(range(11, 22))


for i in range(0, len(census_df_postcode_agg)):
    data = census_df_postcode_agg.iloc[i]
    #print(census_df_postcode_agg.iloc[i])

    # interpolation 
    interpolated_dict = dict()
    interpolated_dict['date']= [f'20{year}' for year in new_years]

    for feature in impute_features:
        init_y = data[[f'{feature}_{init_years[0]}', f'{feature}_{init_years[1]}', f'{feature}_{init_years[2]}']].tolist()
        #print(init_y)

        interp_func = interp1d(init_years, init_y)
        new_y = list(interp_func(new_years))
        #print(new_y)

        interpolated_dict[feature] = new_y

    # creation of new dataframe
    interpolated_df = pd.DataFrame(interpolated_dict)
    
    # model
    interpolated_df['date'] = pd.to_datetime(interpolated_df['date'])
    interpolated_df = interpolated_df.set_index('date')
    interpolated_df = interpolated_df.asfreq('YS') # frequency = year started
    interpolated_df = interpolated_df.sort_index()
    
    steps = 2
    data_train = interpolated_df[:int(0.8 * (len(interpolated_df)))]
    data_test = interpolated_df[int(0.8 * (len(interpolated_df))):]

    # fit model 
    model = VAR(endog = data_train)
    model_fit = model.fit()

    prediction = model_fit.forecast(model_fit.y, steps = len(data_test))



  



    break 



AttributeError: 'VARResults' object has no attribute 'y'